In [2]:

import os

In [3]:
%pwd

'c:\\Users\\Omar\\Desktop\\Omar_Files\\Python_Analysis\\EndToEndMLProjectGenderClassification\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'c:\\Users\\Omar\\Desktop\\Omar_Files\\Python_Analysis\\EndToEndMLProjectGenderClassification'

In [12]:

from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir:Path
    test_data_arr_path:Path
    model_path:Path
    metrics_file_name:Path 
    target_column:str
    all_params:dict

In [13]:
from EndToEndMLProjectGenderClassification.constants import *
from EndToEndMLProjectGenderClassification.utils.common import read_yaml,create_directories,save_json

In [14]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH,
                 schema_filepath=SCHEMA_FILE_PATH) -> None:
        
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        self.schema=read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self)-> ModelEvaluationConfig:
        config=self.config.model_evaluation
        params=self.params.GradientBoostingClassifier
        schema=self.schema

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_arr_path=config.test_data_arr_path,
            model_path=config.model_path,
            metrics_file_name=config.metrics_file_name,
            all_params=params,
            target_column=schema.TARGET_COLUMN

        )

        return model_evaluation_config

In [15]:
import os
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from urllib.parse import urlparse
import joblib,json
from sklearn.metrics import accuracy_score,f1_score,recall_score,precision_score


In [16]:
class ModelEvaluation:
    def __init__(self,config:ModelEvaluationConfig):
        self.config= config

    def evaluation_metrics(self,actual,pred):
        acc=accuracy_score(actual,pred) 
        preci=precision_score(actual,pred) 
        re_call=recall_score(actual,pred) 
        f1=f1_score(actual,pred) 
        return acc,preci,re_call,f1
    
    def save_results(self):
        with open(self.config.test_data_arr_path, 'rb') as f:
            test_data = np.load(f)
        #test_data=pd.read_csv(self.config.test_data_path)
        model=joblib.load(self.config.model_path)

        x_test,y_test=(
            test_data[:,:-1],test_data[:,-1]
        )

        prediction=model.predict(x_test)

        (acc,preci,re_call,f1)=self.evaluation_metrics(y_test,prediction)

        scors={"accuracy":acc,"precision":preci,"recall":re_call,"f1score":f1}

        save_json(path=Path(self.config.metrics_file_name),data=scors)

In [17]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.save_results()
except Exception as e:
    raise e

[2024-09-10 20:55:43,823: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-09-10 20:55:43,826: INFO: common: yaml file: params.yaml loaded successfully]
[2024-09-10 20:55:43,831: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-09-10 20:55:43,832: INFO: common: created directory at: artifacts]
[2024-09-10 20:55:43,834: INFO: common: created directory at: artifacts/model_evaluation]
[2024-09-10 20:55:43,877: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]
